In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [1]:
import tushare as ts

ts.set_token('b44a79bfc0d0985920d0328c722c4b32c64a6f4ea73d6937469fcd8c')
pro = ts.pro_api()

In [7]:
import xlrd
import pandas as pd
from openpyxl import load_workbook

excel_name = r'/Users/lsrsusan/Desktop/code/python数据分析/dsa5205/DSA5205.xls'
# 写入文件位置，*处填入文件位置

Wb = xlrd.open_workbook('DSA5205.xls') 
# 获取工作表中所有的表格

Sheets = Wb.sheet_names()
# 获取工作表中所有表格名

Merge_data = pd.DataFrame()
# 建立空DataFrame，存储数据

for i in range(len(Sheets)):  
# 循环遍历所有Sheets
    
    df = pd.read_excel(excel_name, sheet_name=i)
    df1= df[list(df.columns)]
    df1['Quarter']=Sheets[i]
    Merge_data = Merge_data.append(df1)


/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_data.append(df1)
/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_data.append(df1)
/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_data.append(df1)
/var/folders/tj/wlk3qbws6g1g7_m2qp7sndkm0000gn/T/ipykernel_25255/4205159957.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Merge_data = Merge_d

In [11]:
Merge_data[Merge_data['index']=="AAPL"]

,index,INDEX BLOOMBERG,BS_TOT_ASSET,TOTAL_EQUITY,RETURN_ON_ASSET,RETURN_ON_CAP,RETURN_COM_EQY,Quarter
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2021Q1
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2021Q2
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2021Q3
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2021Q4
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2022Q1
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2022Q2
138,AAPL,AAPL US EQUITY,NaN,NaN,NaN,NaN,NaN,2022Q3


In [12]:
msft = yf.Ticker("YI AAPL")

# get stock info
msft.info

{'regularMarketPrice': None, 'preMarketPrice': None, 'logo_url': ''}

In [13]:
msft.quarterly_balancesheet

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [30]:
df = pro.us_tradecal(start_date='20210101', end_date='20220731')

In [31]:
df.head()

,cal_date,is_open,pretrade_date
0,20220731,0,20220728
1,20220730,0,20220728
2,20220729,1,20220728
3,20220728,1,20220727
4,20220727,1,20220726


In [32]:
df=df[df['is_open']==1]
df=df.reset_index(drop=True)
df['cal_date']

In [37]:
from datetime import datetime
import math

In [33]:
now = datetime.datetime(2015, 1, 1, 0, 0, 0, 0)
present_quarter = int(math.ceil(now.month/3.))
completed_quarter = (present_quarter - 1) if present_quarter != 1 else 4

In [42]:
df['cal_date']=[datetime.strptime(i,"%Y%m%d") for i in list(df['cal_date'].values)]

In [58]:
str(df.iloc[0,0].year)+"1"

'20221'

In [60]:
df['Quarter']=df['cal_date'].map(lambda x:str(x.year)+"Q"+str(math.ceil(x.month/3)))

In [63]:
df=df.iloc[:,[0,-1]]

In [68]:
df.rename(columns={'cal_date':'date'},inplace=True)

In [70]:
df1=pd.merge(df,Merge_data,on="Quarter",how='right')

In [74]:
df1=df1.set_index("date")

In [75]:
df1=df1.sort_index()

In [87]:
df1.reset_index(inplace=True)

In [88]:
df1.head()

,date,Quarter,index,INDEX BLOOMBERG,BS_TOT_ASSET,TOTAL_EQUITY,RETURN_ON_ASSET,RETURN_ON_CAP,RETURN_COM_EQY
0,2021-01-04,2021Q1,BLIN,BLIN US EQUITY,11.603,3.255,-8.3966,NaN,-49.9127
1,2021-01-04,2021Q1,MGYR,MGYR US EQUITY,741.784,58.201,0.4302,2.9056,5.2436
2,2021-01-04,2021Q1,ANAB,ANAB US EQUITY,396.418,381.943,-7.2703,NaN,-7.6247
3,2021-01-04,2021Q1,RCKY,RCKY US EQUITY,449.084,183.867,7.2326,NaN,13.9507
4,2021-01-04,2021Q1,ALTR,ALTR US EQUITY,817.913,399.813,-0.2795,-1.1080,-0.5723


In [90]:
df1=df1.set_index(['date', 'index'])

In [91]:
df1.to_csv("financial_quality_factor.csv")